# 멀티 - GPU 예제

- **데이터 병렬처리** (Data Parallelism)는 Mini-batch를 여러 개의 더 작은 Mini-batch로 자르고 각각의 작은 미니배치를 병렬적으로 연산하는 것을 의미

- 데이터 병렬처리는 `torch.nn.DataParallel`을 사용하여 구현할 수 있다
- `DataParallel`로 감쌀 수 있는 모듈은 배치 차원(batch dimension)에서 여러 GPU로 병렬 처리할 수 있다

---

## DataParallel

In [10]:
import torch
import torch.nn as nn

class DataParallelModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.block1 = nn.Linear(10, 20)
        
        #wrap block2 in DataParallel
        self.block2 = nn.Linear(20, 20)
        self.block2 = nn.DataParallel(self.block2) #감싸준다
        
        self.block3 = nn.Linear(20, 20)
        
        
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        return x

## DataParallel이 구현된 기본형(primitive):

- 일반적으로 Pytroch의 nn.parallel 기본형은 독립적으로 사용할 수 있다.
- 간단한 MPI류의 기본형을 구현해보면 다음과 같다

    - 복제 (replicate) : 여러기기에 모듈을 복제
    - 분산 (scatter) : 첫번째 차원에서 입력을 분산
    - 수집 (gather) : 첫번째 차원에서 입력을 수집하고 합친다
    - 병렬적용(parallel_apply) : 이미 분산된 입력의 집합을 이미 분산된 모델의 집합에 적용한다

In [12]:
def data_parallel(module, input, device_ids, output_device=None):
    if not device_ids:
        return module(input)

    if output_device is None:
        output_device = device_ids[0]

    replicas = nn.parallel.replicate(module, device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    replicas = replicas[:len(inputs)]
    outputs = nn.parallel.parallel_apply(replicas, inputs)
    return nn.parallel.gather(outputs, output_device)

## 모델 일부는 CPU, 일부는 GPU에서 

In [11]:
device = torch.device("cuda:0")

class DistributedModel(nn.Module):

    def __init__(self):
        super().__init__(
            embedding=nn.Embedding(1000, 10),
            rnn=nn.Linear(10, 10).to(device),
        )

    def forward(self, x):
        # CPU에서 연산합니다.
        x = self.embedding(x)

        # GPU로 보냅니다.
        x = x.to(device)

        # GPU에서 연산합니다.
        x = self.rnn(x)
        return x

---

# Refereence

https://tutorials.pytorch.kr/beginner/former_torchies/parallelism_tutorial.html